<br>Лабороторна робота номер 2
<br>Частина 2
<br>Створіть програму, яка дозволить користувачам малювати графік функції гармоніки (функція виду y(t) = A*sin(ωt +φ)) з накладеним шумом та надавати можливість змінювати параметри гармоніки та шуму за допомогою інтерактивного інтерфейсу, що містить слайдери, кнопки та чекбокси. Зашумлену гармоніку відфільтруйте за допомогою фільтру на вибір, порівняйте результат. Використовуйте будь-яку бібліотеку на ваш вибір – matplotlib, seaborn, plotly тощо.

Завантаження потрібних бібліотек

In [25]:
# harmonic_gui.py
import sys
import numpy as np
from scipy import signal
from PyQt5 import QtWidgets, QtCore
from PyQt5.QtCore import Qt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure



Задання паратметрів за замовчуванням

In [26]:
# ---------- Default / initial parameters ----------
DEFAULTS = {
    "amplitude": 1.0,
    "frequency": 5.0,        # Hz
    "phase": 0.0,            # radians
    "noise_mean": 0.0,
    "noise_variance": 0.1,   # variance (σ^2)
    "duration": 1.0,         # seconds
    "fs": 1000,              # sampling rate (Hz)
    "filter_type": "lowpass",
    "filter_order": 4,
    "filter_cutoff1": 10.0,  # Hz (for lowpass/highpass – single cutoff; for bandpass use cutoff1 & cutoff2)
    "filter_cutoff2": 50.0,
    "show_noise": True,
    "show_filtered": True
}



Функція що генерує чисту гармоніку для масиву t

In [27]:
# ---------- Utility functions ----------
def harmonic(t, A, freq, phase):
    return A * np.sin(2 * np.pi * freq * t + phase)



Функція що генерує вектор ґаусового шуму з mean і стандартним відхиленням sigma. rng — об’єкт np.random.default_rng() 

In [28]:
def generate_noise(rng, size, mean, variance):
    sigma = np.sqrt(max(variance, 0.0))
    return rng.normal(loc=mean, scale=sigma, size=size)

Функція що переводить частоти зрізу у нормалізовані згідно з частотою Найквіста (nyq = fs/2) і повертає коефіцієнти b, a фільтра Butterworth.

In [29]:

def design_filter(filter_type, order, cutoff1, cutoff2, fs):
    nyq = 0.5 * fs
    if filter_type == "lowpass":
        Wn = cutoff1 / nyq
        b, a = signal.butter(order, Wn, btype='low')
    elif filter_type == "highpass":
        Wn = cutoff1 / nyq
        b, a = signal.butter(order, Wn, btype='high')
    elif filter_type == "bandpass":
        Wn = [min(cutoff1, cutoff2)/nyq, max(cutoff1, cutoff2)/nyq]
        b, a = signal.butter(order, Wn, btype='band')
    else:
        raise ValueError("Unknown filter type")
    return b, a


filtfilt — zero-phase filtering (обробка вперед-назад) — усуває фазову деформацію.
Якщо filtfilt не спрацює (наприклад, сигнал занадто короткий або коефіцієнти некоректні), робимо lfilter як запасний варіант (але lfilter дає фазову зміну).

In [30]:
def apply_filter(b, a, data):
    # zero-phase filtering
    try:
        return signal.filtfilt(b, a, data)
    except Exception as e:
        # filtfilt can fail for very short signals or unstable filters; fallback to lfilter
        return signal.lfilter(b, a, data)

Створення інтерфейса

In [31]:
# ---------- Main Qt Application ----------
class HarmonicApp(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Harmonic with Noise — Interactive Filter Demo")
        self.setGeometry(100, 100, 1100, 700)

        # Time base
        self.fs = DEFAULTS["fs"]
        self.duration = DEFAULTS["duration"]
        self.t = np.linspace(0, self.duration, int(self.fs * self.duration), endpoint=False)

        # RNG for noise; store noise vector and last noise params
        self.rng = np.random.default_rng()
        self.noise = None
        self.noise_params = {"mean": None, "variance": None}

        # Current parameter state
        self.params = DEFAULTS.copy()

        # UI layout
        main_layout = QtWidgets.QHBoxLayout(self)

        # Left: plot
        plot_layout = QtWidgets.QVBoxLayout()
        self.fig = Figure(figsize=(6,4))
        self.canvas = FigureCanvas(self.fig)
        plot_layout.addWidget(self.canvas)
        self.ax = self.fig.add_subplot(111)
        self.ax.set_xlabel("t (s)")
        self.ax.set_ylabel("y(t)")
        self.ax.grid(True)

        main_layout.addLayout(plot_layout, stretch=3)

        # Right: controls
        controls_layout = QtWidgets.QVBoxLayout()

        # amplitude
        controls_layout.addWidget(QtWidgets.QLabel("<b>Harmonic parameters</b>"))
        self.amp_spin = self._make_double_spin("Amplitude (A)", DEFAULTS["amplitude"], 0.0, 10.0, 0.1, controls_layout)
        self.freq_spin = self._make_double_spin("Frequency (Hz)", DEFAULTS["frequency"], 0.1, 200.0, 0.1, controls_layout)
        self.phase_spin = self._make_double_spin("Phase (rad)", DEFAULTS["phase"], -2*np.pi, 2*np.pi, 0.1, controls_layout)

        # noise controls
        controls_layout.addWidget(QtWidgets.QLabel("<b>Noise parameters</b>"))
        self.noise_mean_spin = self._make_double_spin("Noise mean", DEFAULTS["noise_mean"], -5.0, 5.0, 0.01, controls_layout)
        self.noise_var_spin = self._make_double_spin("Noise variance (σ²)", DEFAULTS["noise_variance"], 0.0, 5.0, 0.01, controls_layout)

        # noise show checkbox
        self.noise_chk = QtWidgets.QCheckBox("Show noisy signal")
        self.noise_chk.setChecked(DEFAULTS["show_noise"])
        controls_layout.addWidget(self.noise_chk)

        # filter controls
        controls_layout.addSpacing(6)
        controls_layout.addWidget(QtWidgets.QLabel("<b>Filter parameters (Butterworth)</b>"))
        self.filter_type_combo = QtWidgets.QComboBox()
        self.filter_type_combo.addItems(["lowpass", "highpass", "bandpass"])
        self.filter_type_combo.setCurrentText(DEFAULTS["filter_type"])
        controls_layout.addWidget(QtWidgets.QLabel("Filter type"))
        controls_layout.addWidget(self.filter_type_combo)

        self.filter_order_spin = QtWidgets.QSpinBox()
        self.filter_order_spin.setRange(1, 10)
        self.filter_order_spin.setValue(DEFAULTS["filter_order"])
        controls_layout.addWidget(QtWidgets.QLabel("Filter order"))
        controls_layout.addWidget(self.filter_order_spin)

        self.cutoff1_spin = self._make_double_spin("Cutoff 1 (Hz)", DEFAULTS["filter_cutoff1"], 0.1, self.fs/2-1, 0.1, controls_layout)
        self.cutoff2_spin = self._make_double_spin("Cutoff 2 (Hz) (for bandpass)", DEFAULTS["filter_cutoff2"], 0.1, self.fs/2-1, 0.1, controls_layout)

        self.filtered_chk = QtWidgets.QCheckBox("Show filtered signal")
        self.filtered_chk.setChecked(DEFAULTS["show_filtered"])
        controls_layout.addWidget(self.filtered_chk)

        # Reset button
        self.reset_btn = QtWidgets.QPushButton("Reset")
        controls_layout.addWidget(self.reset_btn)

        # Spacer
        controls_layout.addStretch()

        # Instructions
        instr = QtWidgets.QTextEdit()
        instr.setReadOnly(True)
        instr.setFixedHeight(200)
        instr.setHtml(self._instructions_html())
        controls_layout.addWidget(instr)

        main_layout.addLayout(controls_layout, stretch=2)

        # Connections
        # Harmonic params -> update but keep noise unless noise params changed
        self.amp_spin.valueChanged.connect(self.on_harmonic_param_changed)
        self.freq_spin.valueChanged.connect(self.on_harmonic_param_changed)
        self.phase_spin.valueChanged.connect(self.on_harmonic_param_changed)

        # Noise params -> regenerate noise and update
        self.noise_mean_spin.valueChanged.connect(self.on_noise_param_changed)
        self.noise_var_spin.valueChanged.connect(self.on_noise_param_changed)
        self.noise_chk.stateChanged.connect(self.update_plot)

        # Filter params -> update filtered signal
        self.filter_type_combo.currentTextChanged.connect(self.update_plot)
        self.filter_order_spin.valueChanged.connect(self.update_plot)
        self.cutoff1_spin.valueChanged.connect(self.update_plot)
        self.cutoff2_spin.valueChanged.connect(self.update_plot)
        self.filtered_chk.stateChanged.connect(self.update_plot)

        # Reset
        self.reset_btn.clicked.connect(self.on_reset)

        # Initialize noise
        self._ensure_noise_vector()  # uses current noise params
        self.update_plot()

    def _instructions_html(self):
        return """
        <h3>Інструкція</h3>
        <ul>
        <li>Керуйте параметрами гармоніки: Амплітуда, Частота, Фаза — значення відразу застосовуються.</li>
        <li>Змінюйте параметри шуму (середнє/дисперсія). Якщо змінюєте параметри шуму — шум генерується наново.</li>
        <li>Якщо змінити тільки параметри гармоніки (наприклад частоту), то шум залишається тим же, що й раніше.</li>
        <li>Виберіть тип фільтра (lowpass/highpass/bandpass), порядок та частоти зрізу. Фільтр застосовується до поточної зашумленої гармоніки.</li>
        <li>Чекбокси дозволяють вмикати/вимикати відображення зашумленої та відфільтрованої кривих. Кнопка Reset відновлює початкові параметри.</li>
        </ul>
        """

    def _make_double_spin(self, label, default, minimum, maximum, step, layout):
        layout.addWidget(QtWidgets.QLabel(label))
        spin = QtWidgets.QDoubleSpinBox()
        spin.setRange(minimum, maximum)
        spin.setSingleStep(step)
        spin.setDecimals(4 if step<0.01 else 3 if step<0.1 else 2)
        spin.setValue(default)
        layout.addWidget(spin)
        return spin

    def on_harmonic_param_changed(self, *   _):
        # Update stored harmonic params; do not regenerate noise
        self.params["amplitude"] = float(self.amp_spin.value())
        self.params["frequency"] = float(self.freq_spin.value())
        self.params["phase"] = float(self.phase_spin.value())
        # do not touch noise vector
        self.update_plot()

    def on_noise_param_changed(self, *_):
        # Update noise params and regenerate noise vector
        self.params["noise_mean"] = float(self.noise_mean_spin.value())
        self.params["noise_variance"] = float(self.noise_var_spin.value())
        self._ensure_noise_vector(force=True)
        self.update_plot()

    def _ensure_noise_vector(self, force=False):
        mean = float(self.noise_mean_spin.value())
        var = float(self.noise_var_spin.value())
        size = self.t.size
        if force or (self.noise is None) or (self.noise_params["mean"] != mean) or (self.noise_params["variance"] != var):
            # regenerate noise
            self.noise = generate_noise(self.rng, size, mean, var)
            self.noise_params["mean"] = mean
            self.noise_params["variance"] = var

    def compute_signals(self):
        A = float(self.amp_spin.value())
        freq = float(self.freq_spin.value())
        phase = float(self.phase_spin.value())
        clean = harmonic(self.t, A, freq, phase)

        # ensure noise vector up-to-date w.r.t noise params
        self._ensure_noise_vector(force=False)
        noisy = clean + self.noise
        return clean, noisy

    def update_plot(self, *_):
        clean, noisy = self.compute_signals()
        self.ax.clear()
        self.ax.grid(True)
        self.ax.set_xlabel("t (s)")
        self.ax.set_ylabel("y(t)")

        # plot clean always
        self.ax.plot(self.t, clean, label="Clean harmonic", linewidth=1.5)

        # plot noisy if enabled
        if self.noise_chk.isChecked():
            self.ax.plot(self.t, noisy, alpha=0.7, label="Noisy harmonic")

        # apply filter and plot if enabled
        if self.filtered_chk.isChecked():
            ftype = self.filter_type_combo.currentText()
            order = int(self.filter_order_spin.value())
            c1 = float(self.cutoff1_spin.value())
            c2 = float(self.cutoff2_spin.value())
            # protect cutoff ranges
            max_cut = 0.5 * self.fs - 1e-6
            c1 = max(0.001, min(c1, max_cut))
            c2 = max(0.001, min(c2, max_cut))
            try:
                b, a = design_filter(ftype, order, c1, c2, self.fs)
                filtered = apply_filter(b, a, noisy)
                self.ax.plot(self.t, filtered, linestyle='--', linewidth=1.5, label="Filtered")
            except Exception as e:
                # show error text on plot
                self.ax.text(0.05, 0.9, f"Filter error: {e}", transform=self.ax.transAxes, color='red')
        self.ax.legend(loc='upper right')
        self.canvas.draw_idle()

    def on_reset(self):
        # restore initial values
        self.amp_spin.setValue(DEFAULTS["amplitude"])
        self.freq_spin.setValue(DEFAULTS["frequency"])
        self.phase_spin.setValue(DEFAULTS["phase"])
        self.noise_mean_spin.setValue(DEFAULTS["noise_mean"])
        self.noise_var_spin.setValue(DEFAULTS["noise_variance"])
        self.filter_type_combo.setCurrentText(DEFAULTS["filter_type"])
        self.filter_order_spin.setValue(DEFAULTS["filter_order"])
        self.cutoff1_spin.setValue(DEFAULTS["filter_cutoff1"])
        self.cutoff2_spin.setValue(DEFAULTS["filter_cutoff2"])
        self.noise_chk.setChecked(DEFAULTS["show_noise"])
        self.filtered_chk.setChecked(DEFAULTS["show_filtered"])

        # regenerate noise to initial noise
        self._ensure_noise_vector(force=True)
        self.update_plot()



In [32]:
def main():
    app = QtWidgets.QApplication(sys.argv)
    win = HarmonicApp()
    win.show()
    sys.exit(app.exec_())

if __name__ == "__main__":
    main()

SystemExit: 0